In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### 网络模型的创建步骤
创建模型有 2 个要素：构建子模块和拼接子模块

### 总结
```
一个 module 里可包含多个子 module。比如 LeNet 是一个 Module，里面包括多个卷积层、池化层、全连接层等子 module
一个 module 相当于一个运算，必须实现 forward() 函数
每个 module 都有 8 个字典管理自己的属性

In [5]:
class LeNet(nn.Module):
    # 子模块创建
    def __init__(self, classes):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, classes)
    # 子模块拼接
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

### 模型容器
```
除了上述的模块之外，还有一个重要的概念是模型容器 (Containers)，常用的容器有 3 个，这些容器都是继承自nn.Module。
    nn.Sequetial：按照顺序包装多个网络层
    nn.ModuleList：像 python 的 list 一样包装多个网络层，可以迭代
    nn.ModuleDict：像 python 的 dict一样包装多个网络层，通过 (key, value) 的方式为每个网络层指定名称。

### nn.Sequetial
```
在传统的机器学习中，有一个步骤是特征工程，我们需要从数据中认为地提取特征，然后把特征输入到分类器中预测。在深度学习的时代，特征工程的概念被弱化了，特征提取和分类器这两步被融合到了一个神经网络中。在卷积神经网络中，前面的卷积层以及池化层可以认为是特征提取部分，而后面的全连接层可以认为是分类器部分。比如 LeNet 就可以分为特征提取和分类器两部分，这 2 部分都可以分别使用 nn.Seuqtial 来包装。

In [6]:
class LeNetSequetial(nn.Module):
    def __init__(self, classes):
        super(LeNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.AvgPool2d(2, 2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.AvgPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        x = self.classifier(x)
        return x

In [7]:
class LeNetSequentialOrderDict(nn.Module):
    def __init__(self, classes):
        super(LeNetSequentialOrderDict, self).__init__()

        self.features = nn.Sequential(OrderedDict({
            'conv1': nn.Conv2d(3, 6, 5),
            'relu1': nn.ReLU(inplace=True),
            'pool1': nn.MaxPool2d(kernel_size=2, stride=2),

            'conv2': nn.Conv2d(6, 16, 5),
            'relu2': nn.ReLU(inplace=True),
            'pool2': nn.MaxPool2d(kernel_size=2, stride=2),
        }))

        self.classifier = nn.Sequential(OrderedDict({
            'fc1': nn.Linear(16*5*5, 120),
            'relu3': nn.ReLU(),

            'fc2': nn.Linear(120, 84),
            'relu4': nn.ReLU(inplace=True),

            'fc3': nn.Linear(84, classes),
        }))
        ...
        ...
        ...

### 总结
```
nn.Sequetial是nn.Module的容器，用于按顺序包装一组网络层，有以下两个特性。
顺序性：各网络层之间严格按照顺序构建，我们在构建网络时，一定要注意前后网络层之间输入和输出数据之间的形状是否匹配
自带forward()函数：在nn.Sequetial的forward()函数里通过 for 循环依次读取每个网络层，执行前向传播运算。这使得我们我们构建的模型更加简洁

### nn.ModuleList
```
nn.ModuleList是nn.Module的容器，用于包装一组网络层，以迭代的方式调用网络层，主要有以下 3 个方法：
append()：在 ModuleList 后面添加网络层
extend()：拼接两个 ModuleList
insert()：在 ModuleList 的指定位置中插入网络层
```

In [9]:
class ModuleList(nn.Module):
    def __init__(self):
        super(ModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(20)])

    def forward(self, x):
        for i, linear in enumerate(self.linears):
            x = linear(x)
        return x


net = ModuleList()

print(net)

fake_data = torch.ones((10, 10))

output = net(fake_data)

print(output)

ModuleList(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
    (5): Linear(in_features=10, out_features=10, bias=True)
    (6): Linear(in_features=10, out_features=10, bias=True)
    (7): Linear(in_features=10, out_features=10, bias=True)
    (8): Linear(in_features=10, out_features=10, bias=True)
    (9): Linear(in_features=10, out_features=10, bias=True)
    (10): Linear(in_features=10, out_features=10, bias=True)
    (11): Linear(in_features=10, out_features=10, bias=True)
    (12): Linear(in_features=10, out_features=10, bias=True)
    (13): Linear(in_features=10, out_features=10, bias=True)
    (14): Linear(in_features=10, out_features=10, bias=True)
    (15): Linear(in_features=10, out_features=10, bias=Tru

### nn.ModuleDict
```
nn.ModuleDict是nn.Module的容器，用于包装一组网络层，以索引的方式调用网络层，主要有以下 5 个方法：
clear()：清空  ModuleDict
items()：返回可迭代的键值对 (key, value)
keys()：返回字典的所有 key
values()：返回字典的所有 value
pop()：返回一对键值，并从字典中删除

In [10]:
class ModuleDict(nn.Module):
    def __init__(self):
        super(ModuleDict, self).__init__()
        self.choices = nn.ModuleDict({
            'conv': nn.Conv2d(10, 10, 3),
            'pool': nn.MaxPool2d(3)
        })

        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'prelu': nn.PReLU()
        })

    def forward(self, x, choice, act):
        x = self.choices[choice](x)
        x = self.activations[act](x)
        return x


net = ModuleDict()

fake_img = torch.randn((4, 10, 32, 32))

output = net(fake_img, 'conv', 'relu')
# output = net(fake_img, 'conv', 'prelu')
print(output)

tensor([[[[5.1309e-01, 8.7948e-02, 0.0000e+00,  ..., 0.0000e+00,
           5.3402e-01, 0.0000e+00],
          [2.4881e-01, 2.5132e-01, 0.0000e+00,  ..., 0.0000e+00,
           9.3012e-01, 0.0000e+00],
          [1.3450e-01, 0.0000e+00, 8.5232e-01,  ..., 9.7109e-01,
           9.3308e-01, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 3.8770e-01,  ..., 0.0000e+00,
           6.4879e-02, 5.8475e-01],
          [4.8913e-01, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],

         [[1.2024e+00, 4.5478e-01, 0.0000e+00,  ..., 0.0000e+00,
           3.1379e-01, 4.0610e-01],
          [0.0000e+00, 5.3054e-01, 5.9884e-01,  ..., 5.3529e-01,
           7.5935e-01, 3.3993e-01],
          [0.0000e+00, 8.4214e-01, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 1.0221e-01],
          ...,
          [0.0000e+00, 0.0000e+00, 6.0060e-01,  ..., 6.8416

### 容器总结
```
nn.Sequetial：顺序性，各网络层之间严格按照顺序执行，常用于 block 构建，在前向传播时的代码调用变得简洁
nn.ModuleList：迭代行，常用于大量重复网络构建，通过 for 循环实现重复构建
nn.ModuleDict：索引性，常用于可选择的网络层

### PyTorch 中的 AlexNet
```
AlexNet 是 Hinton 和他的学生等人在 2012 年提出的卷积神经网络，以高出第二名 10 多个百分点的准确率获得 ImageNet 分类任务冠军，从此卷积神经网络开始在世界上流行，是划时代的贡献。
AlexNet 特点如下：
采用 ReLU 替换饱和激活 函数，减轻梯度消失
采用 LRN (Local Response Normalization) 对数据进行局部归一化，减轻梯度消失
采用 Dropout 提高网络的鲁棒性，增加泛化能力
使用 Data Augmentation，包括 TenCrop 和一些色彩修改
```
![AlexNet](https://image.zhangxiann.com/20200614162004.png "AlexNet")

In [11]:
class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x